In [6]:
import json
from collections import Counter

hub_bl = {'лог компании', 'рная дыра', 'пиарюсь'}
def remove_bl_hubs_from_hubs(doc: dict):
    doc['Hubs'] = [h for h in doc['Hubs'] if all([h_bl not in h for h_bl in hub_bl])]
    return doc

def divide_by_total_count(word_count: dict):
    val_sum = sum(word_count.values())
    for k,v in word_count.items():
        word_count[k] = v/val_sum
    return word_count

probs = sc.textFile('hdfs://master:54310/clean'). \
    map(lambda line: remove_bl_hubs_from_hubs(json.loads(line))). \
    map(lambda doc: (doc['Hubs'], Counter(doc['Text']))). \
    flatMap(lambda x: [(h, x[1]) for h in x[0]]). \
    reduceByKey(lambda a,b: a+b). \
    map(lambda x: (x[0], divide_by_total_count(dict(x[1])))). \
    collectAsMap()
    
from math import log
def calc_log_prob(label: str, words: list):
    word_probs = probs[label]
    return sum([log(word_probs[word]) for word in set(words) if word in word_probs]) 

def choose_max_prob_hub(doc: list, hubs: list):
    if len(hubs) == 1:
        return hubs[0]
    else:
        return max([(h, calc_log_prob(h, doc)) for h in hubs], key=lambda x: x[1])[0]
    
sc.textFile('hdfs://master:54310/clean'). \
    map(lambda line: remove_bl_hubs_from_hubs(json.loads(line))). \
    filter(lambda doc: doc['Hubs']). \
    map(lambda doc: {'Id': doc['Id'],
                     'Features': doc['Text'],
                     'Label': choose_max_prob_hub(doc['Text'], doc['Hubs'])}). \
    map(lambda x: json.dumps(x)). \
    repartition(4). \
    saveAsTextFile('hdfs://master:54310/single-label')